In [1]:
from mmseg.datasets.viperSeq import ViperSeqDataset
from mmseg.datasets.viper import ViperDataset
from torch.utils.data import DataLoader
from functools import partial
from mmcv.parallel import collate
from mmseg.core.evaluation.metrics import flow_prop_iou

/srv/flash1/skareer6/miniconda3/envs/mmseg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset settings
dataset_type = 'ViperDataset'
data_root = '/srv/share4/datasets/VIPER/'

#imagenet values
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

#crop size from the da-vsn paper code
crop_size = (720, 1280)

In [4]:
# train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations'),
#     dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
#     dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
#     dict(type='RandomFlip', prob=0.5),
#     dict(type='PhotoMetricDistortion'),
#     dict(type='Normalize', **img_norm_cfg),
#     dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_semantic_seg']),
# ]
train_pipeline = {
    "im_load_pipeline": [
        dict(type='LoadImageFromFile'),
        dict(type='LoadAnnotations'),
    ],
    "load_no_ann_pipeline": [
        dict(type='LoadImageFromFile'),
    ],
    "load_flow_pipeline": [
        dict(type='LoadFlowFromFile'),
    ],
    "shared_pipeline": [
        # dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
        # dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
        # dict(type='RandomFlip', prob=0.5),
    ],
    "im_pipeline": [
        # dict(type='PhotoMetricDistortion'),
        # dict(type='Normalize', **img_norm_cfg),
        # dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
        dict(type='DefaultFormatBundle'),
        dict(type='Collect', keys=['img', 'gt_semantic_seg'], meta_keys=[]),
    ],
    "flow_pipeline": [
        dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
        dict(type='DefaultFormatBundle'), #I don't know what this is
        dict(type='Collect', keys=['img', 'gt_semantic_seg']),
    ]
}
img_dir = '/srv/share4/datasets/VIPER/'

In [5]:
viper = ViperSeqDataset(
        # type=dataset_type,
        split='splits/train.txt',
        frame_offset=2,
        # flow_dir="/srv/share4/datasets/VIPER_Flowv2/train/flow_occ",
        data_root=data_root,
        img_dir='train/img',
        ann_dir='train/cls',
        pipeline=train_pipeline
)

2022-11-04 12:37:50,432 - mmseg - INFO - Loaded 13367 images
2022-11-04 12:37:50,454 - mmseg - INFO - Loaded 13367 images


PL:  {'im_load_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations'}], 'load_no_ann_pipeline': [{'type': 'LoadImageFromFile'}], 'load_flow_pipeline': [{'type': 'LoadFlowFromFile'}], 'shared_pipeline': [], 'im_pipeline': [{'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg'], 'meta_keys': []}], 'flow_pipeline': [{'type': 'Pad', 'size': (720, 1280), 'pad_val': 0, 'seg_pad_val': 255}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}]}


In [20]:
data_loader = DataLoader(
    viper,
    collate_fn=partial(collate, samples_per_gpu=1)
)
print("made dl")
for i, data in enumerate(data_loader):
    print(data)


made dl
flow off
(1080, 1920, 6)
{'img_metas': DataContainer([[{}]]), 'img': DataContainer([tensor([[[[219, 219, 219,  ...,  34,  55,  42],
          [219, 219, 219,  ...,  38,  63,  70],
          [219, 219, 219,  ...,  30,  28,  32],
          ...,
          [ 78,  80,  86,  ..., 216, 216, 216],
          [ 77,  78,  79,  ..., 216, 216, 216],
          [ 71,  69,  71,  ..., 216, 216, 216]],

         [[196, 196, 196,  ...,  33,  54,  41],
          [196, 196, 196,  ...,  37,  62,  69],
          [196, 196, 196,  ...,  29,  27,  31],
          ...,
          [104, 106, 112,  ..., 240, 240, 240],
          [103, 104, 105,  ..., 240, 240, 240],
          [100,  98,  97,  ..., 240, 240, 240]],

         [[180, 180, 180,  ...,  35,  56,  43],
          [180, 180, 180,  ...,  39,  64,  71],
          [180, 180, 180,  ...,  31,  29,  33],
          ...,
          [118, 120, 126,  ..., 246, 246, 246],
          [119, 120, 121,  ..., 246, 246, 246],
          [115, 113, 113,  ..., 246, 246, 2

KeyboardInterrupt: 